In [1]:
%run ~/relmapping/annot/notebooks/__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/b2/scratch/ahringer/jj374/lab/relmapping


In [2]:
fp_dm_pe = pf('atac_dm', 'tg_pe.bwa_pe.p10_keep.macs2_pe_lt200.concave75_idr', '_0.001.bed', 'atac716_dm')
fp_dm_se = pf('atac_dm', 'tg_pe.bwa_pe.p10_keep.macs2_se_extsize150_shiftm75_keepdup_all.concave_idr', '_0.001.bed', 'atac716_dm')
fp_am_se = pf('atac_am1', 'tg_se.bwa_se.q10_keep.macs2_se_extsize150_shiftm75_keepdup_all.concave150_idr', '_0.001.bed', 'atac716_am')

!wc -l {fp_dm_pe}
!wc -l {fp_dm_se}
!wc -l {fp_am_se}

df_dm_pe = pd.read_csv(fp_dm_pe, sep='\t', comment='#', names=NAMES_BED6)
df_dm_se = pd.read_csv(fp_dm_se, sep='\t', comment='#', names=NAMES_BED6)
df_am_se = pd.read_csv(fp_am_se, sep='\t', comment='#', names=NAMES_BED6)

df_dm_pe['atac_mode'] = list(map(int, df_dm_pe[['start', 'end']].mean(axis=1)))
df_dm_se['atac_mode'] = list(map(int, df_dm_se[['start', 'end']].mean(axis=1)))
df_am_se['atac_mode'] = list(map(int, df_am_se[['start', 'end']].mean(axis=1)))

df_dm_pe['atac_source'] = 'atac_wt_pe'
df_dm_se['atac_source'] = 'atac_wt_se'
df_am_se['atac_source'] = 'atac_glp1_se'

del df_dm_pe['strand']
del df_dm_se['strand']
del df_am_se['strand']

30161 atac716_dm/tg_pe.bwa_pe.p10_keep.macs2_pe_lt200.concave75_idr/atac_dm.tg_pe.bwa_pe.p10_keep.macs2_pe_lt200.concave75_idr_0.001.bed
32442 atac716_dm/tg_pe.bwa_pe.p10_keep.macs2_se_extsize150_shiftm75_keepdup_all.concave_idr/atac_dm.tg_pe.bwa_pe.p10_keep.macs2_se_extsize150_shiftm75_keepdup_all.concave_idr_0.001.bed
29370 atac716_am/tg_se.bwa_se.q10_keep.macs2_se_extsize150_shiftm75_keepdup_all.concave150_idr/atac_am1.tg_se.bwa_se.q10_keep.macs2_se_extsize150_shiftm75_keepdup_all.concave150_idr_0.001.bed


In [3]:
df_dm_pe.to_csv('annot/S1_accessible_sites/atac_wt_pe.bed', sep='\t', header=False, index=False)
df_dm_se.to_csv('annot/S1_accessible_sites/atac_wt_se.bed', sep='\t', header=False, index=False)
df_am_se.to_csv('annot/S1_accessible_sites/atac_glp1_se.bed', sep='\t', header=False, index=False)
!wc -l annot/S1_accessible_sites/atac_wt_pe.bed
!wc -l annot/S1_accessible_sites/atac_wt_se.bed
!wc -l annot/S1_accessible_sites/atac_glp1_se.bed

30160 annot/S1_accessible_sites/atac_wt_pe.bed
32441 annot/S1_accessible_sites/atac_wt_se.bed
29369 annot/S1_accessible_sites/atac_glp1_se.bed


In [4]:
# All peaks in wt_pe
!cp annot/S1_accessible_sites/atac_wt_pe.bed annot/S1_accessible_sites/atac_wt_glp1.bed
!wc -l annot/S1_accessible_sites/atac_wt_glp1.bed
# Add wt_se peaks not in wt_pe
!bedtools subtract -A -a annot/S1_accessible_sites/atac_wt_se.bed -b annot/S1_accessible_sites/atac_wt_pe.bed >> annot/S1_accessible_sites/atac_wt_glp1.bed
!sort -k 1,1 -k 2,2n -k 3,3n -o annot/S1_accessible_sites/atac_wt_glp1.bed annot/S1_accessible_sites/atac_wt_glp1.bed
!wc -l annot/S1_accessible_sites/atac_wt_glp1.bed
# Add glp1_se peaks not in wt_pe+wt_se
!bedtools subtract -A -a annot/S1_accessible_sites/atac_glp1_se.bed -b annot/S1_accessible_sites/atac_wt_glp1.bed >> annot/S1_accessible_sites/atac_wt_glp1.bed
!sort -k 1,1 -k 2,2n -k 3,3n -o annot/S1_accessible_sites/atac_wt_glp1.bed annot/S1_accessible_sites/atac_wt_glp1.bed
!wc -l annot/S1_accessible_sites/atac_wt_glp1.bed
!sed -i '1i#track gffTags=on' annot/S1_accessible_sites/atac_wt_glp1.bed
!wc -l annot/S1_accessible_sites/atac_wt_glp1.bed

30160 annot/S1_accessible_sites/atac_wt_glp1.bed
35728 annot/S1_accessible_sites/atac_wt_glp1.bed
42245 annot/S1_accessible_sites/atac_wt_glp1.bed
42246 annot/S1_accessible_sites/atac_wt_glp1.bed


In [5]:
# Write .tsv file with all the columns
df_merged = pd.read_csv('annot/S1_accessible_sites/atac_wt_glp1.bed',
            names=['chrom', 'start', 'end', 'name', 'score', 'atac_mode', 'atac_source'], sep='\t', comment='#')

In [6]:
# Peak heights for every hypersensitive site
for stage in itertools.islice(config['stages'], None):
    fp_ = 'atac808_geo/tracks/atac_%s.bw' % (stage,)
    print(stage, os.path.isfile(fp_))
    df_merged['atac_%(stage)s_height' % locals()] = list(map(np.nanmax, yp.read_regions(fp_, df_merged.chrom.tolist(), df_merged.start.tolist(), df_merged.end.tolist())))

wt_emb True
wt_l1 True
wt_l2 True
wt_l3 True
wt_l4 True
wt_ya True
glp1_d1 True
glp1_d2 True
glp1_d6 True
glp1_d9 True
glp1_d13 True


In [7]:
write_gffbed('annot/S1_accessible_sites/S1a_accessible_sites.peak_heights.bed',
    chrom = df_merged['chrom'],
    start = df_merged['start'],
    end = df_merged['end'],
    attr = df_merged[['atac_%s_height' % stage_ for stage_ in config['stages']]],
)

In [8]:
# Max peak height across all wt stages (e.g. select top 10k strong peaks)
df_merged['atac_wt_max_height'] = df_merged[['atac_%(stage)s_height' % locals() for stage in config['stages_wt']]].max(axis=1)
df_merged.loc[df_merged['atac_source'] == 'atac_glp1_se', 'atac_wt_max_height'] = 0.0
df_merged['atac_wt_max_rank'] = df_merged['atac_wt_max_height'].rank(ascending=False)

In [9]:
# Count reads within every hypersensitive site
#for rep in itertools.islice(config['atac_geo_by_rep'], None):
#    step_ = 'tg_se.bwa_se.rm_unmapped.rm_chrM.rm_blacklist.rm_q10.macs2_se_extsize1_keepdup_all'
#    fp_ = pf('atac728_%(rep)s' % locals(), step_, '_treat_pileup.bw', 'atac728')
#    if not (os.path.isfile(fp_)):
#        print(rep + ', skipping')
#        continue
#    print(rep)
#    df_merged['atac_%(rep)s_count' % locals()] = list(map(np.nansum,
#        yp.mread_regions(fp_,
#                        df_merged.chrom.tolist(),
#                        (df_merged.start - 50).tolist(),
#                        (df_merged.end + 50).tolist())
#                                                         ))

In [10]:
# Measure peak height (in raw count coverage) within every hypersensitive site
#for rep in itertools.islice(config['atac_geo_by_rep'], None):
#    step_ = 'tg_se.bwa_se.rm_unmapped.rm_chrM.rm_blacklist.rm_q10.macs2_se_extsize150_shiftm75_keepdup_all_noSPMR'
#    fp_ = pf('atac728_%(rep)s' % locals(), step_, '_treat_pileup.bw', 'atac728')
#    if not (os.path.isfile(fp_)):
#        print(rep + ', skipping')
#        continue
#    print(rep)
#    df_merged['atac_%(rep)s_maxcount' % locals()] = list(map(np.nanmax, 
#        yp.mread_regions(fp_, 
#                     df_merged.chrom.tolist(), 
#                     (df_merged.start - 50).tolist(),
#                     (df_merged.end + 50).tolist())
#                                                            ))

In [11]:
# Write output table
fp_accessible_sites = 'annot/S1_accessible_sites/S1a_accessible_sites.tsv'
df_merged.drop(['name', 'score'], axis=1).to_csv(fp_accessible_sites, sep='\t', index=False)
!wc -l {fp_accessible_sites}

42246 annot/S1_accessible_sites/S1a_accessible_sites.tsv


In [12]:
fp_accessible_sites = 'annot/S1_accessible_sites/S1a_accessible_sites.tsv'
df_accessible_sites = pd.read_csv(fp_accessible_sites, sep='\t')

df_ = pd.read_csv('annot733.germline/atac_merge/atac_merged_germline.tsv', sep='\t')
print('%d germline sites (preliminary)' % (len(df_.query("is_atac_wt_l1_decr")),))
df_accessible_sites['atac_is_germline_specific'] = df_['is_atac_wt_l1_decr']

df_accessible_sites.to_csv('annot/S1_accessible_sites/S1a_accessible_sites_with_germline.tsv', sep='\t', index=False)

837 germline sites (preliminary)
